In [7]:
import geopandas as gpd
import os
import pandas as pd
import matplotlib.pylab as plt
import shapely.speedups
import sqlite3
import folium
%config Completer.use_jedi = False #for auto complete code

In [2]:
file = 'D:/Proyectos_GitHub/umbrales_estaciones/data/Maestro/Maestro.xlsx'
df = pd.read_excel(file)

In [3]:
df

,INVENTARIO_SINCERADO,DZ,NOMBRE_ESTACION,CODIGO,CATEGORIA,CLASIFICACION,TIPO,COMUNICACION,FECHA_DE_INSTALACION,ESTADO_FUNCIONAMIENTO,ESTADO_OPERATIVIDAD,LATITUD,LONGITUD,Lat_Sexagesimal,Lot_Sexagesimal,ALTITUD,DEPARTAMENTO,PROVINCIA,DISTRITO
0,NO,1,ALAMOR,200322,CONVENCIONAL,HLM,HIDROLÓGICA,VOZ Y DATA,01/02/17,FUNCIONANDO,OPERATIVO,-4.474008,-80.399152,"-4°28'26.43""","-80°23'56.95""",127.0,PIURA,SULLANA,LANCONES
1,NO,1,ALAMOR,47E0D438,AUTOMATICA,EHA,HIDROLÓGICA,SATELITAL GOES,17/12/14,FUNCIONANDO,OPERATIVO,-4.477490,-80.397895,"-4°28'38.96""","-80°23'52.42""",116.0,PIURA,SULLANA,LANCONES
2,NO,1,BOCATOMA_CHIPILLICO,47E0E1A2,AUTOMATICA,EHA,HIDROLÓGICA,SATELITAL GOES,05/04/15,FUNCIONANDO,OPERATIVO,-4.714203,-80.118248,"-4°42'51.13""","-80°7'5.69""",340.0,PIURA,PIURA,LAS LOMAS
3,NO,1,BOCATOMA_ZAMBA,4723C424,AUTOMATICA,EHA,HIDROLÓGICA,SATELITAL GOES,13/05/14,FUNCIONANDO,OPERATIVO,-4.664442,-79.897353,"-4°39'51.99""","-79°53'50.47""",633.0,PIURA,AYABACA,PAIMAS
4,NO,1,CABO_INGA,47E01126,AUTOMATICA,EHA,HIDROLÓGICA,SATELITAL GOES,05/12/14,FUNCIONANDO,OPERATIVO,-3.978733,-80.399381,"-3°58'43.44""","-80°23'57.77""",160.0,TUMBES,TUMBES,SAN JACINTO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,NO,13,PUENTE_ISLA_CABANILLAS,210406,CONVENCIONAL,HLM,HIDROLÓGICA,VOZ Y DATA,16/07/94,FUNCIONANDO,OPERATIVO,-15.471833,-70.224020,"-15°28'18.6""","-70°13'26.47""",3838.0,PUNO,SAN ROMAN,JULIACA
251,NO,13,PUENTE_LOROCO_CALLACAME,210502,CONVENCIONAL,HLG,HIDROLÓGICA,VOZ Y DATA,09/07/94,FUNCIONANDO,OPERATIVO,-16.570650,-69.322780,"-16°34'14.34""","-69°19'22.01""",3862.0,PUNO,CHUCUITO,KELLUYO
252,NO,13,PUENTE_RAMIS,210101,CONVENCIONAL,HLG,HIDROLÓGICA,VOZ Y DATA,01/01/56,FUNCIONANDO,OPERATIVO,-15.255309,-69.873786,"-15°15'19.11""","-69°52'25.63""",3825.0,PUNO,HUANCANE,TARACO
253,NO,13,PUENTE_ZAPATILLA,270503,CONVENCIONAL,HLM,HIDROLÓGICA,VOZ Y DATA,01/01/76,FUNCIONANDO,OPERATIVO,-16.138603,-69.578730,"-16°8'18.97""","-69°34'43.43""",3846.0,PUNO,EL COLLAO,ILAVE


In [4]:
points_gdf = gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df.LONGITUD,df.LATITUD),crs='EPSG:4326')

C:\Users\jose\anaconda3\envs\work_env\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


In [5]:
this_map=folium.Map(location=[df.LATITUD.mean(),df.LONGITUD.mean()],zoom_start=6)
def plotDot(df):
    folium.CircleMarker(location=[df.LATITUD,df.LONGITUD],radius=2).add_to(this_map)

df.apply(plotDot, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
250    None
251    None
252    None
253    None
254    None
Length: 255, dtype: object

In [6]:
this_map

In [14]:
clima =gpd.read_file('../data/shp/clasif_clima_peru/clasif_clima_1981_2010.shp')

In [15]:
shapely.speedups.enable() 
# enable shapely.speedups which makes queries running faster.
sub_df=[]
for i in range(clima.shape[0]):
    
    pip_mask = points_gdf.within(clima.loc[i, 'geometry']) 
    # selecting the polygon's geometry field to filter out points that 
    # are not overlaid
    pip_data = df.loc[pip_mask]
    
    pip_data['COD_Clima']= clima.CODIGO[i]
    
    sub_df.append(pip_data)
    
maestro = pd.concat((sub_df))

C:\Users\jose\anaconda3\envs\work_env\lib\site-packages\shapely\speedups\__init__.py:50: RuntimeWarning: shapely.speedups not available
  warnings.warn("shapely.speedups not available", RuntimeWarning)
C:\Users\jose\AppData\Local\Temp\ipykernel_13780\2335432169.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pip_data['COD_Clima']= clima.CODIGO[i]


In [16]:
df = pd.concat((sub_df))

In [17]:
df

,INVENTARIO_SINCERADO,DZ,NOMBRE_ESTACION,CODIGO,CATEGORIA,CLASIFICACION,TIPO,COMUNICACION,FECHA_DE_INSTALACION,ESTADO_FUNCIONAMIENTO,...,LATITUD,LONGITUD,Lat_Sexagesimal,Lot_Sexagesimal,ALTITUD,DEPARTAMENTO,PROVINCIA,DISTRITO,geometry,COD_Clima
51,NO,3,EL_TAMBO,211003,CONVENCIONAL,HLM,HIDROLÓGICA,SIN_DATO,2022-04-08 00:00:00,FUNCIONANDO,...,-7.573976,-78.706554,"-7°34'26.31""","-78°42'23.59""",688.0,LA LIBERTAD,GRAN CHIMÚ,MARMOT,POINT (-78.70655 -7.57398),D (i) B'
52,NO,3,EL_TAMBO,47E0A2A8,AUTOMATICA,EHMA,HIDROMETEOROLÓGICA,SATELITAL GOES,06/02/15,FUNCIONANDO,...,-7.573976,-78.706554,"-7°34'26.31""","-78°42'23.59""",688.0,LA LIBERTAD,GRAN CHIMÚ,MARMOT,POINT (-78.70655 -7.57398),D (i) B'
60,NO,3,PUENTE_COINA,211004,CONVENCIONAL,HLM,HIDROLÓGICA,SIN_DATO,SIN_DATO,FUNCIONANDO,...,-7.796364,-78.380122,"-7°47'46.91""","-78°22'48.44""",1823.0,LA LIBERTAD,OTUZCO,USQUIL,POINT (-78.38012 -7.79636),D (i) B'
61,NO,3,PUENTE_COINA,472E74CC,AUTOMATICA,EHMA,HIDROMETEOROLÓGICA,SATELITAL GOES,SIN_DATO,FUNCIONANDO,...,-7.796364,-78.380122,"-7°47'46.91""","-78°22'48.44""",1823.0,LA LIBERTAD,OTUZCO,USQUIL,POINT (-78.38012 -7.79636),D (i) B'
63,NO,3,PUENTE_PALMIRA,211002,CONVENCIONAL,HLM,HIDROLÓGICA,SIN_DATO,SIN_DATO,FUNCIONANDO,...,-7.549189,-78.810514,"-7°32'57.08""","-78°48'37.85""",647.0,LA LIBERTAD,GRAN CHIMÚ,CASCAS,POINT (-78.81051 -7.54919),D (i) B'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,NO,10,PUENTE_PEREZ,230001,CONVENCIONAL,HLM,HIDROLÓGICA,PLANILLA,01/11/17,FUNCIONANDO,...,-9.346940,-75.982500,"-9°20'48.98""","-75°58'57""",675.0,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAÚN,POINT (-75.98250 -9.34694),A (r) A'
213,NO,10,PUENTE_TOCACHE,230700,CONVENCIONAL,HLG,HIDROLÓGICA,VOZ Y DATA,01/01/03,FUNCIONANDO,...,-8.184750,-76.507890,"-8°11'5.1""","-76°30'28.4""",483.0,SAN MARTIN,TOCACHE,TOCACHE,POINT (-76.50789 -8.18475),A (r) A'
218,NO,10,TINGO_MARIA,47270400,AUTOMATICA,EHMA,HIDROMETEOROLÓGICA,SATELITAL GOES,31/12/00,FUNCIONANDO,...,-9.296010,-76.002070,"-9°17'45.64""","-76°0'7.45""",646.0,HUANUCO,LEONCIO PRADO,RUPA-RUPA,POINT (-76.00207 -9.29601),A (r) A'
219,NO,10,TINGO_MARIA,221820,CONVENCIONAL,HLM,HIDROLÓGICA,VOZ Y DATA,22/09/97,PARALIZADA,...,-9.296010,-76.002070,"-9°17'45.64""","-76°0'7.45""",646.0,HUANUCO,LEONCIO PRADO,RUPA-RUPA,POINT (-76.00207 -9.29601),A (r) A'


In [18]:
cuencas=gpd.read_file('../data/shp/cuencas/UH.shp')

In [19]:
shapely.speedups.enable() 
# enable shapely.speedups which makes queries running faster.
sub_df=[]
for i in range(cuencas.shape[0]):
    
    pip_mask = points_gdf.within(cuencas.loc[i, 'geometry']) 
    # selecting the polygon's geometry field to filter out points that 
    # are not overlaid
    pip_data = df.loc[pip_mask]
    
    pip_data['Cuenca']= cuencas.NOMBRE[i]
    
    sub_df.append(pip_data)
    
maestro = pd.concat((sub_df))

C:\Users\jose\anaconda3\envs\work_env\lib\site-packages\shapely\speedups\__init__.py:50: RuntimeWarning: shapely.speedups not available
  warnings.warn("shapely.speedups not available", RuntimeWarning)
C:\Users\jose\AppData\Local\Temp\ipykernel_13780\2079091699.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pip_data['Cuenca']= cuencas.NOMBRE[i]


In [20]:
maestro

,INVENTARIO_SINCERADO,DZ,NOMBRE_ESTACION,CODIGO,CATEGORIA,CLASIFICACION,TIPO,COMUNICACION,FECHA_DE_INSTALACION,ESTADO_FUNCIONAMIENTO,...,LONGITUD,Lat_Sexagesimal,Lot_Sexagesimal,ALTITUD,DEPARTAMENTO,PROVINCIA,DISTRITO,geometry,COD_Clima,Cuenca
4,NO,1,CABO_INGA,47E01126,AUTOMATICA,EHA,HIDROLÓGICA,SATELITAL GOES,05/12/14,FUNCIONANDO,...,-80.399381,"-3°58'43.44""","-80°23'57.77""",160.0,TUMBES,TUMBES,SAN JACINTO,POINT (-80.39938 -3.97873),"C (i , p) A'",Cuenca Tumbes
7,NO,1,EL_TIGRE,200202,CONVENCIONAL,HLG,HIDROLÓGICA,VOZ Y DATA,01/08/64,FUNCIONANDO,...,-80.456850,"-3°46'6.53""","-80°27'24.66""",36.0,TUMBES,TUMBES,SAN JACINTO,POINT (-80.45685 -3.76848),"D (i , p) A'",Cuenca Tumbes
8,NO,1,EL_TIGRE,4724A3F6,AUTOMATICA,EHMA,HIDROMETEOROLÓGICA,SATELITAL GOES,11/08/01,FUNCIONANDO,...,-80.456889,"-3°46'7.32""","-80°27'24.8""",37.0,TUMBES,TUMBES,SAN JACINTO,POINT (-80.45689 -3.76870),"D (i , p) A'",Cuenca Tumbes
15,NO,1,PUENTE_TUMBES,47E024BC,AUTOMATICA,EHA,HIDROLÓGICA,SATELITAL GOES,22/10/14,FUNCIONANDO,...,-80.460872,"-3°34'15.16""","-80°27'39.14""",8.0,TUMBES,TUMBES,TUMBES,POINT (-80.46087 -3.57088),"D (i , p) A'",Cuenca Tumbes
251,NO,13,PUENTE_LOROCO_CALLACAME,210502,CONVENCIONAL,HLG,HIDROLÓGICA,VOZ Y DATA,09/07/94,FUNCIONANDO,...,-69.322780,"-16°34'14.34""","-69°19'22.01""",3862.0,PUNO,CHUCUITO,KELLUYO,POINT (-69.32278 -16.57065),"C (o , i) C'",Cuenca Callaccame
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,NO,3,PUENTE_PALMIRA,211002,CONVENCIONAL,HLM,HIDROLÓGICA,SIN_DATO,SIN_DATO,FUNCIONANDO,...,-78.810514,"-7°32'57.08""","-78°48'37.85""",647.0,LA LIBERTAD,GRAN CHIMÚ,CASCAS,POINT (-78.81051 -7.54919),D (i) B',Cuenca Chicama
64,NO,3,PUENTE_PALMIRA,472E5220,AUTOMATICA,EHMA,HIDROMETEOROLÓGICA,SATELITAL GOES,SIN_DATO,FUNCIONANDO,...,-78.810514,"-7°32'57.08""","-78°48'37.85""",647.0,LA LIBERTAD,GRAN CHIMÚ,CASCAS,POINT (-78.81051 -7.54919),D (i) B',Cuenca Chicama
49,NO,3,CASCABAMBA,201301,CONVENCIONAL,HLM,HIDROLÓGICA,PLANILLA,01/09/12,FUNCIONANDO,...,-78.733769,"-7°22'59.96""","-78°44'1.57""",3331.0,CAJAMARCA,CONTUMAZA,CONTUMAZA,POINT (-78.73377 -7.38332),"B (o , i) B'",Cuenca Chicama
68,NO,3,SALINAR,211001,AUTOMATICA,EHA,HIDROLÓGICA,SATELITAL GOES,23/12/13,FUNCIONANDO,...,-78.961092,"-7°39'38.5""","-78°57'39.93""",346.0,LA LIBERTAD,ASCOPE,ASCOPE,POINT (-78.96109 -7.66069),E (d) B',Cuenca Chicama


In [22]:
conn = sqlite3.connect('../data/DZ/niveles.db')
maestro.drop(['geometry'],axis=1).to_sql(name = 'Maestro',con=conn, if_exists='replace',index=False)
conn.commit()
conn.close()

In [23]:
maestro.Cuenca[1]

'Cuenca Chira'